In [9]:
# 1. IMPORTS
# ============================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score
import joblib

In [10]:
# 2. LOAD DATA
# ============================================
df = pd.read_csv("C:/Users/ACER/OneDrive/Desktop/bank.csv")

In [5]:
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Churn
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:

# Fix column names (your exact columns without spaces)
df.columns = [
    'rownumber', 'customerid', 'surname', 'creditscore', 'geography', 
    'gender', 'age', 'tenure', 'balance', 'numofproducts', 'hascrcard', 
    'isactivemember', 'estimatedsalary', 'churn'
]

print(f"Dataset loaded: {df.shape}")
print("Churn unique values:", df['churn'].unique())
print("Churn dtype:", df['churn'].dtype)

Dataset loaded: (10000, 14)
Churn unique values: [1 0]
Churn dtype: int64


In [12]:
def preprocess_churn_target(series):
    s = pd.to_numeric(series, errors='coerce')
    if not set(s.dropna().unique()).issubset({0, 1}):
        s_str = series.astype(str).str.strip().str.lower()
        y = pd.Series(0, index=series.index)
        y[s_str.isin(['1', 'yes', 'true', 'y', 'churn', 'exited'])] = 1
        return y.astype(int)
    return s.fillna(0).astype(int)

y = preprocess_churn_target(df['churn'])
print("Cleaned churn class balance:")
print(y.value_counts(normalize=True))


Cleaned churn class balance:
churn
0    0.7963
1    0.2037
Name: proportion, dtype: float64


In [13]:
# 3. Features (drop IDs)
features = [
    'creditscore', 'geography', 'gender', 'age', 'tenure', 
    'balance', 'numofproducts', 'hascrcard', 'isactivemember', 'estimatedsalary'
]

X = df[features].copy()
print(f"Features prepared: {X.shape}")

Features prepared: (10000, 10)


In [20]:
# 4. Numeric vs categorical
numeric_features = ['creditscore', 'age', 'tenure', 'balance', 'numofproducts', 'estimatedsalary','hascrcard', 'isactivemember']
categorical_features = ['geography', 'gender']
df['hascrcard'] = df['hascrcard'].map({1: "Yes", 0: "No"})
df['isactivemember'] = df['isactivemember'].map({1: "Yes", 0: "No"})


In [15]:
# 5. Production pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), categorical_features)
    ]
)

banking_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=200, 
        max_depth=10,
        min_samples_split=5,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ))
])

In [16]:
# 6. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n🔄 Training: {X_train.shape[0]} samples...")
banking_pipeline.fit(X_train, y_train)


🔄 Training: 8000 samples...


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['creditscore', 'age',
                                                   'tenure', 'balance',
                                                   'numofproducts',
                                                   'estimatedsalary',
                                                   'hascrcard',
                                                   'isactivemember']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['geography', 'gender'])])),
                ('classifier',
                 RandomForestClassifier(class_weight='balanced', max_depth=10,
                                        min_samples_split=5, n_estimators=200,
                                        n_jobs=-1, random_state=42))])

In [17]:
# 7. Evaluate
y_pred = banking_pipeline.predict(X_test)

# ROBUST proba extraction
proba_output = banking_pipeline.predict_proba(X_test)
print(f"Proba shape: {proba_output.shape}")

if proba_output.shape[1] == 2:
    # Normal binary case
    y_pred_proba = proba_output[:, 1]
    auc_score = roc_auc_score(y_test, y_pred_proba)
else:
    # Single class case - use decision function or fallback
    print("⚠️ Single class detected, using accuracy only")
    y_pred_proba = None
    auc_score = 0.0

accuracy = banking_pipeline.score(X_test, y_test)

print(f"\n📈 RESULTS")
print(f"✅ Accuracy: {accuracy:.3f}")
if auc_score > 0:
    print(f"✅ ROC-AUC:  {auc_score:.3f}")

print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred))

Proba shape: (2000, 2)

📈 RESULTS
✅ Accuracy: 0.845
✅ ROC-AUC:  0.865

📊 Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.89      0.90      1593
           1       0.61      0.67      0.64       407

    accuracy                           0.84      2000
   macro avg       0.76      0.78      0.77      2000
weighted avg       0.85      0.84      0.85      2000



In [18]:
# 8. Save model
joblib.dump(banking_pipeline, "banking_churn_pipeline.pkl")
print(f"\n💾 SAVED: banking_churn_pipeline.pkl")


💾 SAVED: banking_churn_pipeline.pkl


In [19]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(
    banking_pipeline,
    X, y,
    cv=5,
    scoring=['accuracy', 'roc_auc']
)

print("CV accuracy:", cv_results['test_accuracy'].mean())
print("CV ROC-AUC:", cv_results['test_roc_auc'].mean())


CV accuracy: 0.8411000000000002
CV ROC-AUC: 0.8619295913915668
